In [1]:
import os
import cv2
import keras
import joblib
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import os
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications import VGG16, VGG19, ResNet50, InceptionV3, DenseNet121, MobileNetV2
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization
from tensorflow.keras.models import Model
import shutil
# from google.colab import drive
# drive.mount('/content/drive',force_remount=True)

2025-04-05 22:14:15.418998: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-05 22:14:15.426600: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743902055.434527   61173 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743902055.436948   61173 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-05 22:14:15.445128: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

# Functions

In [2]:
def get_relative_file_paths(folder_path):

    """
    Gets a list of relative paths to all files within a given folder.

    Args:
        folder_path (str): The path to the folder.

    Returns:
        list: A list of relative file paths.
    """

    file_paths = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            file_paths.append(file_path)
    return file_paths



# Definitions

In [3]:
#METHOD = 'kmeans'
METHOD = 'agglomerative'
# METHOD = 'hdbscan'

NUM_CLUSTERS = 15 #5 # Elejido a mano

MODEL = 'VGG16'

INPUT_SHAPE = (128, 128, 3)
IMAGE_DIR = "../../output/cropped_cells_full_v4/media/"
ENCODER_PATH = "/content/drive/My Drive/Investigacion/UTN/GIAR/Results/Autoencoder/encoder_SSIM_MAE_Bparams.keras"
output_path_dataset = f"../../media/{MODEL}_{METHOD}_v0"


#### Paths

In [4]:
if MODEL == "AutoEncoder":
    encoder = keras.saving.load_model(ENCODER_PATH)
else:
    # Load pre-trained models
    if MODEL == 'VGG16':
        preprocess_input = keras.applications.vgg16.preprocess_input
        encoder = VGG16(weights='imagenet', include_top=False, input_shape=INPUT_SHAPE)
    elif MODEL == 'VGG19':
        preprocess_input = keras.applications.vgg19.preprocess_input
        VGG19(weights='imagenet', include_top=False, input_shape=INPUT_SHAPE)
    elif MODEL == 'ResNet50':
        preprocess_input = tf.keras.layers.Identity
        ResNet50(weights='imagenet', include_top=False, input_shape=INPUT_SHAPE)
    elif MODEL == 'InceptionV3':
        preprocess_input = keras.applications.inception_v3.preprocess_input
        InceptionV3(weights='imagenet', include_top=False, input_shape=INPUT_SHAPE)
    elif MODEL == 'DenseNet121':
        preprocess_input = tf.keras.layers.Identity
        DenseNet121(weights='imagenet', include_top=False, input_shape=INPUT_SHAPE)
    elif MODEL == 'MobileNetV2':
        preprocess_input = tf.keras.layers.Identity
        MobileNetV2(weights='imagenet', include_top=False, input_shape=INPUT_SHAPE)

    inp = keras.Input(shape=INPUT_SHAPE)
    x = preprocess_input(inp)
    x = encoder(x)
    x = Flatten()(x)
    encoder = Model(inputs=inp, outputs=x)


I0000 00:00:1743902057.486547   61173 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21612 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:01:00.0, compute capability: 8.6


In [5]:
file_count = len(os.listdir(IMAGE_DIR))
print(f"Total de archivos: {file_count}")

Total de archivos: 53633


In [6]:
encoder.output_shape

(None, 8192)

In [7]:
image_paths = sorted(get_relative_file_paths(IMAGE_DIR))

# Load the images

In [8]:
from tqdm.contrib.concurrent import process_map

def load_image (x):
    if MODEL == "AutoEncoder":
        return cv2.imread(x, cv2.IMREAD_GRAYSCALE)
    else:
        return cv2.imread(x)

/home/nicolas/Documentos/UTN/INA/giar_ina_dev/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
import os
print(os.cpu_count())

16


In [10]:
images = process_map(
                load_image,
                image_paths,
                total=len(image_paths),
                max_workers=12,
                chunksize=32,
            )

100%|██████████| 53633/53633 [00:02<00:00, 23113.13it/s]


## Use encoder embeddings as features

In [11]:
#Transform input images for encoder input
resized_images = [cv2.resize(image, INPUT_SHAPE[0:2]) for image in images]
resized_images = np.array(resized_images)

# images = images/255 # Esto esta incluido en el modelo

#resized_images = [np.expand_dims(image, axis=(0, -1)) for image in resized_images]

In [12]:
if MODEL == "AutoEncoder":
    resized_images = resized_images / 255.0
    resized_images = np.expand_dims(resized_images, axis=-1)
resized_images.shape

(53633, 128, 128, 3)

In [13]:

encoder.output_shape

(None, 8192)

In [14]:

# Extract features from encoder


enc_features_array = np.zeros((resized_images.shape[0], encoder.output_shape[-1]))

batch_size = 256
ini = 0
while True:
    start = ini*batch_size
    end = start+batch_size

    if start >= resized_images.shape[0]:
        break

    if end >= resized_images.shape[0]:
        end = resized_images.shape[0]-1


    this_batch = resized_images[start:end]

    enc_features_array[start:end] = encoder.predict(this_batch, verbose=0)

    ini += 1



I0000 00:00:1743902063.516548   61254 service.cc:148] XLA service 0x713e400179e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1743902063.516568   61254 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2025-04-05 22:14:23.522065: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1743902063.549649   61254 cuda_dnn.cc:529] Loaded cuDNN version 90701
I0000 00:00:1743902064.720261   61254 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


In [15]:
if METHOD == 'kmeans':
    kmeans = KMeans(n_clusters=NUM_CLUSTERS, random_state=42)
    kmeans.fit(enc_features_array)

    centroids = kmeans.cluster_centers_
    seleccted_class = -np.ones((len(enc_features_array)), dtype=int)
    for idx, feature in enumerate(enc_features_array):
        dist = 1e99
        for cluster in range(NUM_CLUSTERS):
            t_dist = np.linalg.norm(feature - centroids[cluster])
            if dist > t_dist:
                dist = t_dist
                seleccted_class[idx] = cluster




In [16]:
import gc
import psutil

enc_features_array_norm = [a / (np.linalg.norm(a) + 1e-16) for a in enc_features_array]
print(f"Memory usage before: {psutil.virtual_memory().used / 1024**2:.2f} MB")
images = None
resized_images = None
enc_features_array = None
del enc_features_array
del images
del resized_images
gc.collect()
print(f"Memory usage after: {psutil.virtual_memory().used / 1024**2:.2f} MB")

Memory usage before: 20129.27 MB
Memory usage after: 15672.58 MB


In [ ]:
if METHOD == 'agglomerative':
    from sklearn.cluster import AgglomerativeClustering

    # Aca si o si normalizamos
    # enc_features_array_norm = [a / (np.linalg.norm(a) + 1e-16) for a in enc_features_array]

    # ag_clustering = AgglomerativeClustering
    #     n_clusters = None,
    #     metric = 'euclidean',
    #     linkage = 'ward',
    #     distance_threshold = 1.0,
    #     compute_full_tree = True,
    # )

    ag_clustering = AgglomerativeClustering(
        n_clusters = NUM_CLUSTERS,
        linkage = 'ward',
    )

    clustering = ag_clustering.fit(enc_features_array_norm)
    # seleccted_class = clustering.labels_

joblib.dump(clustering, '../../models/agglomerative_clustering.pkl')

In [ ]:
if METHOD == 'hdbscan':
    from sklearn.cluster import HDBSCAN
    hdb = HDBSCAN()
    clustering = hdb.fit(enc_features_array_norm)
    seleccted_class = clustering.labels_

In [ ]:
np.unique(seleccted_class)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

### Create datasets

In [ ]:
# if not os.path.exists('./output/noise'):
#     os.makedirs('./output/noise')

# if not os.path.exists('./output/cells'):
#     os.makedirs('./output/cells')

#### Get cluster representatrives

In [ ]:
NUM_CLUSTERS = len(np.unique(seleccted_class))
GRID_SIZE = 10
NUM_SHOW = GRID_SIZE*GRID_SIZE

rep_images =[[] for _ in range(NUM_CLUSTERS)]
for cluster in range(NUM_CLUSTERS):
    for idx, label in enumerate(seleccted_class):
        if label >= 0:
            if len(rep_images[label]) < NUM_SHOW:
                rep_images[label].append(resized_images[idx])
                # if label == 1:
                #     print(idx, label)

for rep_idx, rep_list in enumerate(rep_images):
    print(rep_idx, len(rep_list))

0 100
1 100
2 100
3 100
4 100
5 100
6 100
7 100
8 100
9 100
10 100
11 100
12 100
13 100
14 100


In [ ]:
len(rep_images[0])

100

In [ ]:

for rep_idx, rep_list in enumerate(rep_images):

    plt.figure(figsize=(8, 8))
    n = 1
    for image in rep_list:
        plt.subplot(GRID_SIZE,GRID_SIZE,n)
        plt.imshow(image,cmap="gray")
        plt.axis(False)
        n+=1
    plt.suptitle(f"Cluster {rep_idx}", fontsize=20, fontweight="bold")
    plt.tight_layout()

    plt.show()


#### Create noise/cell dataset from predictions

In [ ]:
raise ValueError("Pick a cluster MANUALLY!!!!! (this is to catch \"run all\" executions)")

ValueError: Pick a cluster MANUALLY!!!!! (this is to catch "run all" executions)

In [ ]:
# Manually pick one
CELL_CLUSTERS = [8,11,12,13]
#NOT_CLUSTERS=[1,4,6,8]

In [ ]:
SPLIT = 0.7
realization_samples = dict()
for file in os.listdir(IMAGE_DIR):
    base, realiz, _ = file.split("_")
    if base not in realization_samples.keys():
        realization_samples[base]  = set()
    realization_samples[base].add(realiz)


train_samples = dict()
validation_samples = dict()
for key in realization_samples.keys():
    images_here = len(realization_samples[key])

    train_images_here = int(np.floor(images_here*SPLIT))
    this_train_sample = np.random.choice(list(realization_samples[key]), train_images_here, replace=False)
    if key not in train_samples.keys():
        train_samples[key]  = list()
        validation_samples[key]  = list()
    train_samples[key] = this_train_sample
    validation_samples[key] = [a for a in list(realization_samples[key]) if a not in this_train_sample]

print("Train:")
for key in train_samples.keys():
    print(key, len(train_samples[key]))
print("Validation:")
for key in validation_samples.keys():
    print(key, len(validation_samples[key]))

Train:
004 92
Entrega1 6
005 9
003 57
002 50
001 58
Validation:
004 40
Entrega1 3
005 4
003 25
002 22
001 26


In [ ]:
max_class = 3500



# Random Shuffle
indices = np.arange(seleccted_class.shape[0])
np.random.shuffle(indices)
seleccted_class_shuffle = seleccted_class[indices]
image_paths_shuffle = np.array(image_paths)[indices]




if os.path.exists(output_path_dataset):
    shutil.rmtree(output_path_dataset)
os.makedirs(os.path.join(output_path_dataset, 'train', "not"))
os.makedirs(os.path.join(output_path_dataset, 'train', "cells"))
os.makedirs(os.path.join(output_path_dataset, 'validation', "not"))
os.makedirs(os.path.join(output_path_dataset, 'validation', "cells"))





for idx, cluster in enumerate(seleccted_class_shuffle):
    if cluster >= 0:
        if cluster in CELL_CLUSTERS:
            folder = "cells"
        else:
            folder = "not"

        file = image_paths_shuffle[idx]
        img_name = os.path.basename(file)
        base, realiz, _ = img_name.split("_")

        if realiz in train_samples[base]:
            split = "train"
        else:
            split = "validation"

        if len(os.listdir(os.path.join(output_path_dataset, split,folder))) >= max_class:
            continue


        shutil.copyfile(file, os.path.join(output_path_dataset, split, folder, img_name))



In [ ]:
print(os.path.basename(output_path_dataset))

VGG16_agglomerative_v0


In [ ]:
print(len(os.listdir(os.path.join(output_path_dataset, "train", "not"))))
print(len(os.listdir(os.path.join(output_path_dataset, "train", "cells"))))
print(len(os.listdir(os.path.join(output_path_dataset, "validation", "not"))))
print(len(os.listdir(os.path.join(output_path_dataset, "validation", "cells"))))

3500
3500
3500
3193
